In [1]:
import itertools
import collections
import pathlib

## part 1 ##

Use what is essentially a line-sweep algorithm. Stolen from https://work.njae.me.uk/2021/12/29/advent-of-code-2021-day-22/,
although I didn't steal any of the code there since I don't know any Haskell. I just took the basic idea. A real line-sweep
algorithm would be a lot more complicated than this code actually is. 

The clever part about this is that it focuses on boundaries where something changes. For the smallest example, those
boundaries are one cube thick, so that doesn't save a lot of time, but for the bigger problems that's no longer
the case, so this speeds things up a lot.

In [2]:
smalltestlines = '''on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10'''.splitlines()

In [3]:
testlines = '''on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682'''.splitlines()

In [4]:
partbtestlines = pathlib.Path('day22_sampleb.txt').read_text().splitlines()

In [5]:
puzzlelines = pathlib.Path('day22.txt').read_text().splitlines()

In [6]:
Rule = collections.namedtuple('Rule', ['lvl', 'status', 'xmin', 'xmax', 'ymin', 'ymax', 'zmin', 'zmax'])
def parselines(lines):
    rules = []
    for i,line in enumerate(lines):
        status, locs = line.split()
        xinfo, yinfo, zinfo = locs.split(',')
        xmin, xmax = xinfo[2:].split('..')
        ymin, ymax = yinfo[2:].split('..')
        zmin, zmax = zinfo[2:].split('..')
        rule = Rule(i, status, int(xmin), int(xmax), int(ymin), int(ymax), int(zmin), int(zmax))
        rules.append(rule)
    return rules

In [7]:
offsets = {'x': 2, 'y': 4, 'z': 6}
def get_boundaries(axis, rules):
    offset = offsets[axis]
    bdrys = collections.defaultdict(list)
    for rule in rules:
        bdrys[rule[offset]].append(rule.lvl)
        bdrys[rule[offset+1]+1].append(rule.lvl)
    return bdrys  

In [8]:
def solve(lines):
    rules = parselines(lines)
    on = 0
    zactive = set()
    zbdrys = get_boundaries('z', rules)
    zs = sorted(zbdrys.keys())
    for zlayer in zip(zs, zs[1:]):
        zcurr = zlayer[0]
        zthick = zlayer[1] - zlayer[0]
        for rulenum in zbdrys[zcurr]:
            if rules[rulenum].zmin == zcurr:
                zactive.add(rulenum)
            else:
                zactive.remove(rulenum)
        currzrules = [rules[rule] for rule in zactive]
        xbdrys = get_boundaries('x', currzrules)
        xs = sorted(xbdrys.keys())
        xactive = set()
        for xlayer in zip(xs, xs[1:]):
            xcurr = xlayer[0]
            xthick = xlayer[1] - xlayer[0]
            for rulenum in xbdrys[xcurr]:
                if rules[rulenum].xmin == xcurr:
                    xactive.add(rulenum)
                else:
                    xactive.remove(rulenum)
            currxrules = [rules[rule] for rule in xactive]
            ybdrys = get_boundaries('y', currxrules)
            ys = sorted(ybdrys.keys())
            yactive = set()
            for ylayer in zip(ys, ys[1:]):
                ycurr = ylayer[0]
                ythick = ylayer[1] - ylayer[0]
                for rulenum in ybdrys[ycurr]:
                    if rules[rulenum].ymin == ycurr:
                        yactive.add(rulenum)
                    else:
                        yactive.remove(rulenum)
                active_rules = zactive & xactive & yactive
                if active_rules:
                    active = max(active_rules)
                    if rules[active].status == 'on':
                        on += zthick*xthick*ythick
    return on

In [9]:
solve(smalltestlines)

39

I really don't want to write code just to leave out the cubes outside -50 to 50. Just pass in the appropriate lines.

In [10]:
solve(testlines[:-2])

590784

In [11]:
solve(puzzlelines[:20])

615869

## part 2 ##

In [12]:
solve(partbtestlines)

2758514936282235

In [13]:
solve(puzzlelines)

1323862415207825